## Categorical Focal Loss
In this assignment we will implement a categorical focal loss function with "L1" and "L2" regularization for multi-class classification problems.\
Focal Loss have several applications in problems which have inbalance datasets such as Object Detection:
you can learn more about this loss function here:
https://medium.com/swlh/focal-loss-what-why-and-how-df6735f26616

In [2]:
import tensorflow as tf

## Focal Loss Formula:
$$
FL(y_{true}, y_{pred}) = - \alpha * y_{true} * (1 - y_{pred})^ γ * log(y_{pred}) \\
l1(y_{true}, y_{pred}) = ∑|y_{pred}| \\
l2(y_{true}, y_{pred}) = \sum (y_{pred})^2 \\
total-loss = FL + l1_w * l1 + l2_2 * l2
$$

In [7]:
class CategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2, l1=0.01, l2=0.01, **kwargs):
        super(CategoricalFocalLoss, self).__init__(**kwargs)
        self.alpha = alpha
        self.gamma = gamma
        self.l1 = l1
        self.l2 = l2

    def call(self, y_true, y_pred):
        epsilon = 1e-8
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Compute categorical focal loss
        cross_entropy = -self.alpha*y_true * tf.math.pow(1.0 - y_pred, self.gamma) * tf.math.log(y_pred)
        focal_loss = tf.reduce_sum(cross_entropy, axis=-1)

        # Compute L1 regularization
        l1_loss = self.l1 * tf.reduce_sum(tf.abs(y_pred - y_true))

        # Compute L2 regularization
        l2_loss = self.l2 * tf.reduce_sum(tf.square(y_pred - y_true))

        # Total loss
        total_loss = focal_loss + l1_loss + l2_loss

        return total_loss


In [8]:

def build_model(dense_units, input_shape=(224, 224) + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [9]:
import tensorflow_datasets as tfds
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes
model = build_model(dense_units=256)

# Compile with losses and metrics
model.compile(optimizer='adam', loss=CategoricalFocalLoss(), metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., tf.cast(tf.one_hot(features['label'], depth=2), tf.float32)

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
model.fit(dataset, epochs=10)

Epoch 1/10
727/727 [==============================] - 38s 50ms/step - loss: 0.4166 - accuracy: 0.6968
Epoch 2/10
727/727 [==============================] - 38s 52ms/step - loss: 0.3095 - accuracy: 0.7997
Epoch 3/10
727/727 [==============================] - 36s 50ms/step - loss: 0.2304 - accuracy: 0.8604
Epoch 4/10
727/727 [==============================] - 36s 50ms/step - loss: 0.1537 - accuracy: 0.9145
Epoch 5/10
727/727 [==============================] - 36s 50ms/step - loss: 0.1009 - accuracy: 0.9453
Epoch 6/10
727/727 [==============================] - 36s 50ms/step - loss: 0.0633 - accuracy: 0.9663
Epoch 7/10
727/727 [==============================] - 37s 51ms/step - loss: 0.0410 - accuracy: 0.9788
Epoch 8/10
727/727 [==============================] - 37s 50ms/step - loss: 0.0248 - accuracy: 0.9877
Epoch 9/10
727/727 [==============================] - 37s 51ms/step - loss: 0.0177 - accuracy: 0.9911
Epoch 10/10
727/727 [==============================] - 36s 50ms/step - loss: 0.019